# Simple UDP application
This notebook shows a simple UDP application with Network Support from Vitis using GT Kernels.
There are 4 Kernels:
* CMAC: provides the translation between physical signals to AXI4-Stream interface
* Network layer: provides a bridge between raw Ethernet packets and the application using UDP as transport layer
    * The UDP module has a 16-entry table with socket information that needs to be filled in before running
* krnl_mm2s: reads data from memory and packetize it setting tdest and tlast appropriately
* krnl_s2mm: read data from the stream and copy it to memory

![](./../img/udp_network_basic.jpg)

## Import packages and program FPGA

In [1]:
# Import packages and configure FPGA
import pynq
import numpy as np
# import thread module 
from _thread import *
import threading 
import socket
currentDevice = pynq.Device.devices[2]
ol = pynq.Overlay('../build_dir.intf0.xilinx_u280_xdma_201920_3/xup_vitis_networking.xclbin',device=currentDevice)

## Run ping to get ARP set up
Some of the attempts will fail but the remaining should work.
* This notebook assumes that the local IP of your NIC is 192.168.0.25 and it was already configured

In [2]:
!ping -c 5 192.168.0.5

PING 192.168.0.5 (192.168.0.5) 56(84) bytes of data.
64 bytes from 192.168.0.5: icmp_seq=3 ttl=128 time=0.111 ms
64 bytes from 192.168.0.5: icmp_seq=4 ttl=128 time=0.048 ms
64 bytes from 192.168.0.5: icmp_seq=5 ttl=128 time=0.037 ms

--- 192.168.0.5 ping statistics ---
5 packets transmitted, 3 received, 40% packet loss, time 3999ms
rtt min/avg/max/mdev = 0.037/0.065/0.111/0.033 ms


## Create software table with valid sockets and software socket variables

In [3]:
socketType = np.dtype([('theirIP', np.uint32), ('theirPort', np.uint16), ('myPort', np.uint16), ('valid', np.bool)])
sockets = np.zeros(16, dtype=socketType)
sockets[0] = (0xC0A80019, 50446, 60133, True)
sockets[1] = (0xC0A80019, 38746, 62781, True)

FPGA_IP = '192.168.0.5'
FPGA_PORT = sockets[1]['myPort']
SW_PORT = sockets[1]['theirPort']

## Transfer socket table to FPGA

In [4]:
from pynq import MMIO
def initSocketTable(ol, sockets, debug = False):
    global currentDevice
    network_address = ol.ip_dict["networklayer_1"]["phys_addr"]
    udp_address_offset = ol.ip_dict["networklayer_1"]["registers"]["udp_offset"]["address_offset"]
    udp_phy_address = network_address + udp_address_offset
    udp_handler = MMIO(udp_phy_address, 0x1000, device=currentDevice)
    # Get maximum number of sockets in hardware
    numSocketsHW = udp_handler.read(0x210)
    if (numSocketsHW is not len(sockets)):
        raise Exception('Socket list length ({}) is not equal to maximum number of sockets in hardware ({})'.format(len(sockets),numSocketsHW))
    
    
    for i in range(len(sockets)):
        ti_offset = 0x10 + i*8
        tp_offset = ti_offset + len(sockets) * 8
        mp_offset = ti_offset + len(sockets) * 8 * 2
        v_offset  = ti_offset + len(sockets) * 8 * 3
        
        udp_handler.write(ti_offset, int(sockets[i]['theirIP']))
        udp_handler.write(tp_offset, int(sockets[i]['theirPort']))
        udp_handler.write(mp_offset, int(sockets[i]['myPort']))
        udp_handler.write(v_offset , int(sockets[i]['valid']))
        

    if debug:
        print("Number of Sockets: {}" .format(udp_handler.read(0x2d0)))
        for i in range(len(sockets)):
            #print(sockets[i])
            ti_offset = 0x10 + i*8
            tp_offset = ti_offset + len(sockets) * 8
            mp_offset = ti_offset + len(sockets) * 8 * 2
            v_offset  = ti_offset + len(sockets) * 8 * 3

            ti = udp_handler.read(ti_offset)
            tp = udp_handler.read(tp_offset)
            mp = udp_handler.read(mp_offset)
            v  = udp_handler.read(v_offset)
            
            print("HW socket table[{:3d}], ti: 0x{:08x}\ttp: {:5d}\tmp: {:5d}\tv: {:1d}".format(i,ti,tp,mp,v))
    
  
initSocketTable(ol,sockets, True)

Number of Sockets: 0
HW socket table[  0], ti: 0xc0a80019	tp: 50446	mp: 60133	v: 1
HW socket table[  1], ti: 0xc0a80019	tp: 38746	mp: 62781	v: 1
HW socket table[  2], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[  3], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[  4], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[  5], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[  6], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[  7], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[  8], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[  9], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[ 10], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[ 11], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[ 12], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[ 13], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[ 14], ti: 0x00000000	tp:     0	mp:     0	v: 0
HW socket table[ 15], ti: 0x00000000	tp:     0	mp

## Get design handlers

In [5]:
network_layer = ol.networklayer_1
mm2s = ol.krnl_mm2s_1
s2mm = ol.krnl_s2mm_1

## Bind software socket

In [6]:
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # UDP
sock.bind(('', SW_PORT))

## Allocate buffers and initialize input

In [7]:
size = 1408 * 100
shape = (size,1)

mm2s_buf = pynq.allocate(shape, dtype=np.uint8, target=ol.HBM0)
s2mm_buf = pynq.allocate(shape, dtype=np.uint8, target=ol.HBM0)
mm2s_buf[:] = np.random.randint(low=0, high=((2**8)-1), size=shape, dtype=np.uint8)

## Move data from the HOST through the network
* Start kernel stream to memory mapped
* Send as many packets as necessary
* The FPGA will receive the data copy it to global memory

In [8]:
s2mm_wh = s2mm.start(s2mm_buf,size)
udp_message_global = np.random.randint(low=0, high=((2**8)-1), size=shape, dtype=np.uint8)
BYTES_PER_PACKET = 1408
num_pkts = size//BYTES_PER_PACKET
for m in range(num_pkts):
    udp_message_local = udp_message_global[(m * BYTES_PER_PACKET) : ((m * BYTES_PER_PACKET) + BYTES_PER_PACKET)]
    sock.sendto(udp_message_local, (FPGA_IP, FPGA_PORT))

### Wait for kernel completion
* Move data from global memory to HOST
* Compare what was sent against what was received and print out result

In [9]:
s2mm_wh.wait()
s2mm_buf.sync_from_device()

msg = "SUCCESS!" if np.array_equal(udp_message_global, s2mm_buf) else "FAILURE!"
print("Host sending data through the network and the host getting data from kernel was a: {}. Total data transmitted {:,} bytes to {}" .format(msg,size,(FPGA_IP, FPGA_PORT)))

Host sending data through the network and the host getting data from kernel was a: SUCCESS!. Total data transmitted 140,800 bytes to ('192.168.0.5', 62781)


## Create threaded function to receive data from the network

In [10]:
print_lock = threading.Lock() 
# thread function 
def socket_receive_threaded(sock, size): 
    BYTES_PER_PACKET = 1408
    shape_global = (size,1)
    shape_local = (BYTES_PER_PACKET,1)
    recv_data_global = np.empty(shape_global, dtype = np.uint8)
    data_partial = np.empty(shape_local, dtype = np.uint8)
    num_it = (size // BYTES_PER_PACKET)
    global mm2s_buf
    sum_bytes = 0
    connection = 'None'
    for m in range(num_it):
        res = sock.recvfrom_into(data_partial) 
        recv_data_global[(m * BYTES_PER_PACKET) : ((m * BYTES_PER_PACKET) + BYTES_PER_PACKET)] = data_partial
        sum_bytes = sum_bytes + int(res[0])
        connection = res[1]
    msg = "SUCCESS!" if np.array_equal(mm2s_buf, recv_data_global) else "FAILURE!"
    print ("Kernel sending data to the network and the host getting data from network was a: {}. Total data received {:,} bytes from {}".format(msg,sum_bytes,connection))


## Move data from the HOST through PCIE to the FPGA which will send it to the network
* Move HOST memory to FPGA global memory
* Lock acquired by client 
* Launch threaded function
* Start kernel memory mapped to stream kernel
* Threaded function will start receiving data and it will print out result once ALL data was collected

In [11]:
mm2s_buf.sync_to_device()
print_lock.acquire() 
start_new_thread(socket_receive_threaded, (sock,size,))
mm2s_wh = mm2s.start(mm2s_buf,size, 1)

Kernel sending data to the network and the host getting data from network was a: SUCCESS!. Total data received 140,800 bytes from ('192.168.0.5', 62781)


## Delete buffers and free Alveo

In [12]:
del mm2s_buf
del s2mm_buf
del udp_message_global
pynq.Overlay.free(ol)